# Generative AI Hackathon

## Introduction

This notebook walks you through the challenge of implementing a **Knowledge Worker** for your organisation using **Generative AI**!

**The challenge:** Decentralized data across internal and external databases results in time wasted as workforce tries to find required information and transform into insights.

➡️ **Your task:** Implement a knowledge worker to enable users in your company to perform Q&A, in natural language, upon a knowledge base.
In this way, you'll centralise company data for easy access in a user-friendly manner, boosting productivity.
As such, you'll create a knowledge worker fine-tuned to your data domain.
This app will only have access to specific knowledge such as public data about your company available on your company's website and unstructured documents (PDF, Word, text ...).

While solving the tasks as instructed in this notebook, you'll familiarise yourself with common concepts and tools for Generative AI including:

- The Open-Source tool LangChain
- Large Language Models (LLMs)
- Text Embeddings and Vector Databases
- Prompts and Prompt Engineering

Ultimately, this notebook details how to get started with LangChain, and walks through setting up a knowledge worker on Google Cloud and Vertex AI.

## Implementing a knowledge worker

When creating a knowledge worker, you recall Large Language Models (LLMs) can be tuned for a variety of tasks such as text summarisation, answering questions, and generating new content (and many more!).
When it comes to tuning approaches, you'll have the choice between:

**A) Zero-shot learning:** Use LLMs directly without providing additional data or fine-tuning the model.

**B) Few-shot learning:** Provide a select number of input examples when using LLM to improve the quality of outputs.

**C) Model Fine-tuning:** Fine-tune certain (or additional) layers in the LLM by training the model on provided training data.

Instead of training LLMs using your own data (ie. fine-tuning), it is far easier and more effective to adapt the LLM to your use-case by prompt engineering only (ie. tuning).
Thus, methods A) and B) are more applicable for creating your first knowledge worker.

A knowledge worker can be approached in two stages:

1. Embedding knowledge from diverse sources.
2. Querying a LLM which is aware of your relevant knowledge to answer questions.

![A typical ingestion chain](https://github.com/teamdatatonic/gen-ai-hackathon/blob/7f37d477b18ace5912d34b0574512559d7a457ed/assets/typical-ingestion-chain.png?raw=true)

First, documents (websites, Word documents, databases, Powerpoints, PDFs, etc.) are loaded and split into chunks. Fragmenting is important for three reasons:

1. There are technical restrictions on how much data (tokens) can be fed into an LLM at once, meaning the context + system prompt + chat history + user prompt must fit within the token limit.
2. Most LLM APIs operate on a per-token pricing model, meaning it is cost-effective to limit the size / amount of data provided to the LLM per query.
3. Contextual information should be relevant to the user query, meaning it is optimal to provide only relevant snippets from documents, making the answer more relevant whilst saving costs as per (1) and (2).

Next, these document shards are embedded within a vector store. Embedding a document means to align it within a mutli-dimension space, which can then be searched according to user queries to find relevant documents. Document relevancy scoring can be as simple as a K-neighbours search, since embedded documents with similarity (as percieved by the LLM embedding model) will be proximate within the search space.

![A typical query chain](https://github.com/teamdatatonic/gen-ai-hackathon/blob/7f37d477b18ace5912d34b0574512559d7a457ed/assets/typical-query-chain.png?raw=true)

Once the vector store is created, a user can query the knowledge base using natural language questions. Relevant documents related to the query are found in the vector store by embedding the user query and finding local documents. These snippets of text are provided to the LLM (alongside the user query, chat history, prompt engineering, etc.) which parses the information to generate an answer.

## Prerequisites

**Authenticate with Google Cloud:**

In [1]:
from google.colab import auth as google_auth
google_auth.authenticate_user()

**Install Python dependencies:**

In [4]:
%pip install --quiet langchain chromadb tiktoken gradio unstructured tqdm google-cloud-aiplatform google-cloud-core

**Restart Python Kernel:**
Ensure that your environment can access the newly installed dependencies.

In [6]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

**Note:** If your kernel doesn't restart automatically, click the "Restart Runtime" button above your notebook.
If you dont see a restart button, go to the "Runtime" toolbar tab then "Restart Runtime". After restarting, continue executing the project from below this cell.

## Setup Projects and Region

**If you don't know your project ID**, try the following:
* Run `gcloud config list`.
* Run `gcloud projects list`.
* See the support page: [Locate the project ID](https://support.google.com/googleapi/answer/7014113)

In [5]:
PROJECT_ID = "dt-langchain-apps-dev" # @param {type:"string"}
LOCATION = "europe-west2"  # @param {type:"string"}
BUCKET = "dt-langchain-apps-dev-zurich-hackathon" # @param {type:"string"}

In [6]:
!gcloud config set project {PROJECT_ID}

Updated property [core/project].


In [7]:
from google.cloud import aiplatform
aiplatform.init(project=PROJECT_ID, location=LOCATION)

## Task 01: Q&A over Documentation with GoogleLLM

Creating a custom knowledge worker is similar to your first step when learning a new programming language.
As such your first challenge is to create a “Hello World” program, however, adapted to LLMs which is way more exciting!

With a few lines of code, you'll:
- Load documents with information about your company
- Create text embeddings from documents
- Storing embedding in a local database
- Use an LLM to answer queries about your company knowledge

**All of these steps can be achieved in a few lines of Python.**

### Download documents

First, we need to collect our documentation.
As we will be constructing our knowledge worker using website data in this first task, we'll first download the files locally.

To get started fast, we've already downloaded some sample website data upfront.
Let's copy the website data from a public Cloud Storage bucket to your local file system:

In [12]:
!gsutil cp gs://{BUCKET}/datatonic.com.tar.gz . && tar -xzf datatonic.com.tar.gz

Copying gs://dt-langchain-apps-dev-zurich-hackathon/datatonic.com.tar.gz...
/ [1 files][  9.2 MiB/  9.2 MiB]                                                
Operation completed over 1 objects/9.2 MiB.                                      


**➡️ Your task:**  Use `wget` to download your company's public website data your local file system.
You can use the shell command below to download the data.

In [13]:
WEBSITE = "https://datatonic.com/" # @param {type:"string"}

In [ ]:
!wget -r -A.html {WEBSITE}

Ensure that you remember where you've stored your company's website data:

In [21]:
LOCAL_FOLDER = "datatonic.com" # @param {type:"string"}

Note how filepaths to the `.html` files form proper URL paths, we'll use this later to reference our answers with proper hyperlinks.

**🎉 Congratulations! 🎉** You've downloaded your company data. Now, you'll create create text embeddings as a next


### Introduction to LangChain

LangChain is a Python framework for developing applications using language models.
It abstracts the connection between applications and LLMs, allowing a loose coupling between code and specific providers like Google PaLM.

LangChain supports [numerous methods](https://python.langchain.com/en/latest/modules/indexes/document_loaders.html) for loading documents.

The downloaded website can then be loaded as documents using the LangChain [`DirectoryLoader`](https://python.langchain.com/en/latest/modules/indexes/document_loaders/examples/directory_loader.html) and [`UnstructuredHTMLLoader`](https://python.langchain.com/en/latest/modules/indexes/document_loaders/examples/html.html) loaders.

**➡️ Your task:** Read the linked resources in the *Introduction to LangChain* and study the following code cells as they provide reusable LangChain code for your knowledge worker.

In [22]:
from langchain.document_loaders import DirectoryLoader, UnstructuredHTMLLoader

def load_documents(source_dir):
    # Load the documentation using a HTML parser
    loader = DirectoryLoader(
        source_dir,
        glob="**/*.html",
        loader_cls=UnstructuredHTMLLoader,
        show_progress=True,
    )
    documents = loader.load()

    return documents


### Creating or loading embeddings

Creating embeddings each time we use our app is time-consuming and expensive.
By persisting the vector store database after embedding, we can load the saved embeddings for use in another session.

Also note the use of the `GoogleLLMEmbeddings()` model.
There are [multiple](https://python.langchain.com/en/latest/modules/models/text_embedding.html?highlight=embedding) text embedding models available, many of which can be directly substituted here.
Remember to add additional environment variables for different API keys, as per each model's documentation.

In [23]:
PERSIST_DIR = "chromadb"

In [24]:
from langchain.vectorstores import Chroma
from langchain.embeddings import VertexAIEmbeddings

def load_embeddings():
    # We use GoogleLLM embeddings model, however other models can be substituted here
    embeddings = VertexAIEmbeddings()

    # Creating embeddings with each re-run is highly inefficient and costly.
    # We instead aim to embed once, then load these embeddings from storage.
    vector_store = Chroma(
        embedding_function=embeddings,
        persist_directory=PERSIST_DIR,
    )

    return vector_store


In [25]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

def embed_documents(embedding, documents):
    # Individual documents will often exceed the token limit.
    # By splitting documents into chunks of 1000 token
    # These chunks fit into the token limit alongside the user prompt
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    texts = text_splitter.split_documents(documents)

    vector_store = Chroma.from_documents(
        documents=texts, embedding=embedding, persist_directory=PERSIST_DIR
    )

    # Persist the ChromaDB locally, so we can reload the script without expensively re-embedding the database
    vector_store.persist()

    return vector_store


In [26]:
def create_embeddings(source_dir):
    documents = load_documents(source_dir=source_dir)

    # We use GoogleLLM embeddings model, however other models can be substituted here
    embedding = VertexAIEmbeddings()

    vector_store = embed_documents(embedding, documents)

    return vector_store


**➡️ Your task:** Run the following cell to create the text embeddings based on your downloaded data.

In [20]:
import os

# If the vectorstore embedding database has been created, load it
if os.path.isdir(PERSIST_DIR):
    print(f"Loading {PERSIST_DIR} as vector store")
    vector_store = load_embeddings()
# If it doesn't exist, create it
else:
    print(f"Creating new vector store in dir {PERSIST_DIR}")
    vector_store = create_embeddings(source_dir=LOCAL_FOLDER)


Creating new vector store in dir chromadb


  0%|          | 0/157 [00:00<?, ?it/s][nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
100%|██████████| 157/157 [01:05<00:00,  2.39it/s]


**🎉 Congratulations! 🎉** You've created text embeddings from your company data and stored them successfully in a local vector database.
Now, you'll shift your focus to implementing the actual LLM by creating a chain using LangChain.

### Creating the Conversational Q&A Chain

In this section, you'll create a chain which will be able to provide an answer given a question from a user.
To understand the purpose of chains, you can read about chains in the [LangChain documentation](https://docs.langchain.com/docs/).

At first, we'll initialise a few hyperparameters for the LLM models which we'll reference later on:

In [27]:
# The 'k' value indicates the number of sources to use per query.
# 'k' as in 'k-nearest-neighbours' to the query in the embedding space.
# 'temperature' is the degree of randomness introduced into the LLM response.
k = 2
temperature = 0.0


#### Prompt engineering

As outlined before, the creation of prompts is essential to adapt LLMs for your given use case.
**Prompt engineering** is a method of zero-shot fine-tuning for large language models.
By prompting a LLM with contextual information about its purpose, the model can simulate a variety of situations, such as a customer assistant chatbot, a document summariser, a translator, etc.

In this use case, we prompt our model to respond as a conversational Q&A chatbot.
Prompt engineering can be especially useful for introducing guard rails to an application - in this template we tell the model to not respond to queries it lacks the information to answer, as users will trust the application to provide factual replies, so rejecting a query is preferable to outputting false information.

You can use the prompt and code cells below for your knowledge worker.

**➡️ Your task:** Study the following code cells as they provide reusable LangChain code for your knowledge worker.
Pay attention to the prompt in the `template` variable.
What elements do you notice in the prompt?
How is the prompt used in the chain?

In [28]:
from langchain.prompts import PromptTemplate

template = """\
You are a helpful chatbot designed to perform Q&A on a set of documents.
Always respond to users with friendly and helpful messages.
Your goal is to answer user questions using relevant sources.

You were developed by Datatonic, and are powered by Google's PaLM-2 model.

In addition to your implicit model world knowledge, you have access to the following data sources:
- Company documentation.

If a user query is too vague, ask for more information.
If insufficient information exists to answer a query, respond with "I don't know".
NEVER make up information.

Chat History:
{chat_history}
Question: {question}
"""

# The PromptTemplate reads input variables (i.e.: 'chat_history', 'question') from the template
SYSTEM_PROMPT = PromptTemplate.from_template(template)


In [29]:
from langchain.chains import ConversationalRetrievalChain
from langchain.llms import VertexAI

def qa_chain():
    # A vector store retriever relates queries to embedded documents
    retriever = vector_store.as_retriever(k=k)

    # The selected GoogleLLM model uses embedded documents related to the query
    # It parses these documents in order to answer the user question.
    # We use the GoogleLLM LLM, however other models can be substituted here
    model = VertexAI(temperature=temperature)

    # A conversation retrieval chain keeps a history of Q&A / conversation
    # This allows for contextual questions such as "give an example of that (previous response)".
    # The chain is also set to return the source documents used in generating an output
    # This allows for explainability behind model output.
    chain = ConversationalRetrievalChain.from_llm(
        llm=model,
        retriever=retriever,
        return_source_documents=True,
        condense_question_prompt=SYSTEM_PROMPT,
    )

    return chain


In [30]:
def q_a(question: str, history: list):
    # map history (list of lists) to expected format of chat_history (list of tuples)
    chat_history = map(tuple, history)

    # Query the LLM to get a response
    # First the Q&A chain will collect documents semantically similar to the question
    # Then it will ask the LLM to use this data to answer the user question
    # We also provide chat history as further context
    response = qa_chain()(
        {
            "question": question,
            "chat_history": chat_history,
        }
    )

    # Format source documents (sources of excerpts passed to the LLM) into links the user can validate
    sources = [
        "[{0}]({0})".format(doc.metadata["source"])
        for doc in response["source_documents"]
    ]

    # Return the LLM answer, and list of sources used (formatted as a string)
    return response["answer"], "\n\n".join(sources)


### Building a user interface

As building a UI is outside of the scope for this hackathon, a templateed UI using [Gradio](https://gradio.app/) is provided for your knowledge worker.

**➡️ Your task:** Execute the cells below to launch the user interface.

In [33]:
def submit(msg, chatbot):
    # First create a new entry in the conversation log
    msg, chatbot = user(msg, chatbot)
    # Then get the chatbot response to the user question
    chatbot = bot(chatbot)
    return msg, chatbot


def user(user_message, history):
    # Return "" to clear the user input, and add the user question to the conversation history
    return "", history + [[user_message, None]]


def bot(history):
    # Get the user question from conversation history
    user_message = history[-1][0]
    # Get the response and sources used to answer the user question
    bot_message, bot_sources = q_a(user_message, history[:-1])

    # Using a template, format the response and sources together
    bot_template = (
        "{0}\n\n<details><summary><b>Sources</b></summary>\n\n{1}</details>"
    )
    # Place the response into the conversation history and return
    history[-1][1] = bot_template.format(bot_message, bot_sources)
    return history


In [ ]:
import gradio as gr

# Build a simple GradIO app that accepts user input and queries the LLM
# Then displays the response in a ChatBot interface, with markdown support.
with gr.Blocks(theme=gr.themes.Base()) as demo:
    # Set a page title
    gr.Markdown("# Custom knowledge worker")
    # Create a chatbot conversation log
    chatbot = gr.Chatbot(label="🤖 knowledge worker")
    # Create a textbox for user questions
    msg = gr.Textbox(
        label="👩‍💻 user input", info="Query information from the custom knowledge base."
    )

    # Align both buttons on the same row
    with gr.Row():
        send = gr.Button(value="Send", variant="primary", size="sm")
        clear = gr.Button(value="Clear History", variant="secondary", size="sm")

    # Submit message on <enter> or clicking "Send" button
    msg.submit(submit, [msg, chatbot], [msg, chatbot], queue=False)
    send.click(submit, [msg, chatbot], [msg, chatbot], queue=False)

    # Clear chatbot history on clicking "Clear History" button
    clear.click(lambda: None, None, chatbot, queue=False)

# Create a queue system so multiple users can access the page at once
demo.queue()
# Launch the webserver locally
demo.launch(share=True, debug=True)


**➡️ Your task:** Use the user interface above (which you can also open in a separate tab given the shareable link above), to query your knowledge base.

Try out a few questions below:

> 👩‍💻: tell me about company x!
>
> 👩‍💻: how can I apply for company x?
> 🦜:  You can install the gcloud CLI by running the command `$ gcloud components update`.


You can also ask general questions since LLMs have been trained on massive amounts of public data:

> 👩‍💻: how to install the gcloud cli
>
> 🦜:  You can install the gcloud CLI by running the command `$ gcloud components update`.
>
> 👩‍💻: that command requires gcloud to be installed, how can I install gcloud initially?
>
> 🦜:  You can install gcloud initially by running the command '`pip install google-cloud`' in your terminal.
>
> 👩‍💻: how can I set the target project
>
> 🦜:  You can set the target project for the gcloud CLI by using the command `$ gcloud config set project my-new-default-project`.
>
> 👩‍💻: what is the gcloud cli?
>
> 🦜:  The gcloud CLI is a command line interface for Google Cloud Platform services.
>
> 👩‍💻: explain the above in more detail
>
> 🦜:  The gcloud CLI is a tool used to authenticate and configure credentials for Google Cloud services. It can be used to change the default project ID, update components, and authenticate the CLI itself.

Since we used a `ConversationalRetrievalChain`, we can also correct the model when it gives the wrong response and prompt it to fix it’s mistake, or ask for further detail on a previous response.

**🎉 Congratulations! 🎉** You've created your first chain using LangChain which you can query for general questions in a user interface.
Let's continue extending your knowledge worker in the next task.

**Note:** Stop the cell which runs the user interface before contuining with the next task.

## Task 2: Extending your knowledge base

As mentioned, it is possible to extend a knowledge base with additional documents.
This is useful for updating a knowledge base with new information without having to re-embed established knowledge from scratch.

If you wanted to build a knowledge worker with another document type, for instance [Microsoft Word](https://python.langchain.com/en/latest/modules/indexes/document_loaders/examples/microsoft_word.html) documents, you would update the `load_documents()` function according to the documentation for that document type loader.

See the following example for loading Word documents:
```python
from langchain.document_loaders import Docx2txtLoader

def load_documents():
    # Load the documentation using a Microsoft Word parser
    loader = Docx2txtLoader("example_data/fake.docx")
    documents = loader.load()

    return documents
```

**Note:** if you're looking to deploy a knowledge worker with several knowledge bases, a [Router Chain](https://python.langchain.com/en/latest/modules/chains/examples/multi_retrieval_qa_router.html), which combines several knowledge workers with discrete knowledge bases into a single chain which selects the best worker for the query.

**➡️ Your task:** Load at least one additional document type such as PDF or Doc.
For that update the `load_documents` function to load `.pdf` or `.pptx` files, or write some logic to load any of these file types depending on an argument `document_type` (try using an `if-elif-else` switch to change which document loader is used) and embed multiple document types within the same vector store.

In [35]:
def create_embeddings(source_dir):
    # TODO: update the code below

    documents = load_documents(source_dir=source_dir)

    # We use GoogleLLM embeddings model, however other models can be substituted here
    embedding = VertexAIEmbeddings()

    vector_store = embed_documents(embedding, documents)

    return vector_store

If you've different sources of data, you'll need to update the way of embedding data as you'll need to loop over a sequence of folders instead of processing a single folder only.

**➡️ Your task:** Extend the `create_embeddings()` function to load multiple knowledge sources.
Update the `source_dir` to `source_dirs`, which accepts a list of folder paths, then iteratively load these folders as documents and embed them in the database.

In [36]:
def load_documents(source_dir):
    # Load the documentation using a HTML parser

    # TODO: update the code below
    loader = DirectoryLoader(
        source_dir,
        glob="**/*.html",
        loader_cls=UnstructuredHTMLLoader,
        show_progress=True,
    )
    documents = loader.load()

    return documents

**➡️ Your task:** Load your text embeddings again after updating the two functions `create_embeddings` and `load_documents` and test your changes directly in the UI.
You can run the cells below to load your embeddings and launch the UI.

In [37]:
import os

# If the vectorstore embedding database has been created, load it
if os.path.isdir(PERSIST_DIR):
    print(f"Loading {PERSIST_DIR} as vector store")
    vector_store = load_embeddings()
# If it doesn't exist, create it
else:
    print(f"Creating new vector store in dir {PERSIST_DIR}")
    vector_store = create_embeddings(source_dir=LOCAL_FOLDER)


Loading chromadb as vector store


In [ ]:
# Create a queue system so multiple users can access the page at once
demo.queue()
# Launch the webserver locally
demo.launch(share=True, debug=True)

**🎉 Congratulations! 🎉** You've extended your knowledge to creating text embedding from a variety of sources - whether it's public data from your company's website or unstructured documents!

**Note:** Stop the cell which runs the user interface before contuining with the next task.

## Task 3: Generating text over a vector index

We can utilise our embedded documents for more than just Q&A.
In tasks 1 and 2, we used the embedded documents as context for answering user queries, but in this task we will use it to generate original content using this knowledge base as a source of information and style.

The concept of this use case is to generate ideas for new blogs, utilising knowledge and style information contained in the existing company website data.
We can use Generativr AI for creative ideation, too!
Let's demonstrates the possibilities for human-computer interaction (HCI) apps in this task.

In [40]:
from langchain.chains import LLMChain


In [50]:
prompt_template = """\
Using the provided context, write the outline of a company blog post.
Include a bullet-point list of the main talking points, and a brief summary of the overall blog.
Context: {context}
Topic: {topic}
"""

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "topic"]
)

model = VertexAI(temperature=0.7)

chain = LLMChain(llm=model, prompt=PROMPT)


In [51]:
def generate_blog_outline(topic: str, k: int):
    # search for 'k' nearest documents related to our topic.
    docs = vector_store.similarity_search(topic, k=k)

    # associate topic with the content of each document to generate inputs
    inputs = [{"context": doc.page_content, "topic": topic} for doc in docs]
    # generate blog outline
    output = chain.apply(inputs)

    return output


**➡️ Your task:** Create ideas for a new blog post.
Try adjusting the title of the post to generate new ideas!

In [56]:
BLOG_TITLE = "Greentonic: Make cloud projects more sustainable" # @param {type:"string"}

In [ ]:
from IPython.display import display, Markdown

# generate variations of blog posts on the topic "Greentonic initiative" based on the 4 most relevant documents
output = generate_blog_outline(BLOG_TITLE, k=4)
markdown = ""
for i, blog in enumerate(output):
  markdown += f"# #{i} {BLOG_TITLE}\n{blog['text']}\n\n"
display(Markdown(markdown))


**➡️ Your task:** Now it's time to change the prompt template to create new content based on your liking.
For that update the `prompt_template` and `temperature`.

In [ ]:
# TODO: setup the prompt, model, and chain to create new types of content.


**🎉 Congratulations! 🎉** You've completed the last task of this hackathon!
Continue with the next section to explore next steps for *your* Generative AI journey on Google Cloud.

# Conclusion

## What have we built?

In this session, we have built a knowledge worker use-case for accessing your complex information using Generative AI.
This concept can be extended into a fully-fledged tool that can unlock the value of your data for customers or internal use.

## Going further..

This workshop has introduced all the LangChain knowledge required to create a knowledge worker.
The next steps for moving this project from development to production are discussed below.

## Decoupling LangChain from Gradio

It is not necessary to run LangChain within a GradI/O app.
Decoupling LangChain into a separate API has several benefits:
1. We can deploy scalable servers / Docker containers
2. Simplified code - a frontend-backend loose coupling can lead to simpler code, which is ease to update and maintain.
3. If a more professional user interface is needed, such as a native React app.
Replacing GradI/O is a straightforward process - FastAPI can be called from javascript, etc., allowing you to move beyond Python frontend frameworks.

An example of this separation can be found on the GitHub repository, using FastAPI to create a simple LangChain API server and Poetry to manage separate server environments.

## Deploying on Google Cloud

Once we have decoupled our frontend and backend code, we can deploy the project onto Google Cloud.

This reference architecture diagrams mirror the flow diagrams we first introducted in the workshop introduction. Using Google Cloud, we can create production pipelines for creating / updating vector databases, and deploy a knowledge worker API (which can be connected to a web UI, Slack bot, etc.).

**Example architecture: Ingestion**
![A typical ingestion chain](https://github.com/teamdatatonic/gen-ai-hackathon/blob/7f37d477b18ace5912d34b0574512559d7a457ed/assets/knowledge-worker-gcp-ingestion-pipeline.png?raw=true)

By creating a pipeline for data ingestion, we can continue to extend the knowledge base of our knowledge worker as you produce new documents and documentation.

**Example architecture: Inference**
![A typical ingestion chain](https://github.com/teamdatatonic/gen-ai-hackathon/blob/7f37d477b18ace5912d34b0574512559d7a457ed/assets/knowledge-worker-gcp-inference-pipeline.png?raw=true)

By creating a pipeline for inference, we can leverage the power of Google Cloud to provide a highly reliable and scalable API that can power a variety of applications.



**🎉 Congratulations! 🎉** You've completed this notebook!
Now it's time to embark your Generative AI journey and ideate about use cases which can benefit your company in conjunction or in addition to your first knowledge worker.